<style>.md-sidebar--secondary {display: none !important;}</style>

# Simple FCFS

This is a very simple network, with only 3 markets, used to demonstrate some features of PassengerSim.

In [ ]:
import passengersim as pax
pax.versions()

This example uses [network/01-base.yaml](./network/01-base.yaml) configuration file.
Within a Jupyter notebook, we can directly initialize a PassengerSim `Simulation` 
instance from this file using the `from_yaml` class constructor:

In [ ]:
sim = pax.Simulation.from_yaml("network/01-base.yaml")

Running the simulation is as simple as calling the `run` command, which runs the simulation and returns a summary output object.

In [ ]:
summary = sim.run()

The contents of the final summary is controlled by `Config.outputs.reports`, which allows the user to add reports for more detail, 
or drop some unneccessary reports to improve runtime (sometimes substantially).

In [ ]:
sim.config.outputs.reports

For this example, several default reports are included, which allows us to access a number of pre-packaged visualizations for the results.

In [ ]:
summary.fig_carrier_revenues()

In [ ]:
summary.fig_carrier_load_factors()

In [ ]:
summary.fig_carrier_mileage()

In [ ]:
summary.fig_fare_class_mix()

All demand is in the lowest fare class, because the simulation has no 
restrictions against customers simply buying the least expensive fare.

In [ ]:
summary.fig_bookings_by_timeframe()

We are not limited to the pre-packaged visualizations. The various summary tables available in the `summary`
object are all just regular pandas DataFrames, so we can use all the usual Python and Pandas tools for analysis.
For example, the `demand_to_come` table summarizes the total demand to come at each timeframe for every simulation sample.

In [ ]:
summary.demand_to_come

With this data, we can do whatever analysis we like.  Here we'll compute the correlation between 
total demands (from the beginning of the booking curve at DCP 63 all the way to the end) of 
different passenger types in different markets.

In [ ]:
summary.demand_to_come[63].unstack(["segment", "orig", "dest"]).corr()

The `summary` object also has a command to dump all the summary tables to an Excel workbook, if you prefer to analyze the results there instead of in Jupyter.

In [ ]:
summary.to_xlsx("outputs/3mkt-01.xlsx")

[Download 3mkt-01.xlsx](./outputs/3mkt-01.xlsx)

## Comparing against Targets

In addition to summary reports for a single run, we can also use PassengerSim's `contrast` package to compare simulation runs to each other, or against exogenously defined target results.

In [ ]:
import targets

target = targets.load(1, sim.config)

In [ ]:
from passengersim import contrast

comps = contrast.Contrast({
    "simulation": summary,
    "target": target,
})

In [ ]:
comps.fig_carrier_revenues()

In [ ]:
comps.fig_bookings_by_timeframe(by_carrier="AL1")

In [ ]:
comps.fig_bookings_by_timeframe(by_carrier=False, by_class=True)

We can look at carrier forecasts of demand on individual legs.

In [ ]:
comps.fig_leg_forecasts(by_flt_no=111, of=["mu", "sigma"])

We can compare the mean and standard deviation of demand to come.

In [ ]:
comps.fig_demand_to_come("mean") | comps.fig_demand_to_come("std")

We can even take arbitrary functions that apply pandas tools, and have them run automatically against multiple summary objects.
For example, we can look at the variance-covariance matrix of aggregate demand by passenger type, and compare those
matrices for both the simulation and the target.

In [ ]:
comps.apply(lambda s: s.aggregate_demand_history(by_segment=True).unstack("segment").cov())